# TensorBoard
In this tutorial, we will implement a convolutional neural network and visualize the graph by using tensorboard.

![alt text](jpg/cnn.jpg)

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


mnist = input_data.read_data_sets("mnist/", one_hot=True)
x_train = mnist.train.images.reshape(-1, 28, 28, 1)
y_train = mnist.train.labels
x_test = mnist.test.images.reshape(-1, 28, 28, 1)
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 28, 28, 1)
y_train:  (55000, 10)
x_test:  (10000, 28, 28, 1)
y_test:  (10000, 10)


## Define Network

In [2]:
def conv2d(x, channel_out, k_w=5, k_h=5, s_w=1, s_h=1, name=None):
    """Computes convolution operation
    
    Args:
        x: input tensor of shape (batch_size, width_in, heigth_in, channel_in)
        channel_out: number of channel for output tensor
        k_w: kernel width size; default is 5
        k_h: kernel height size; default is 5
        s_w: stride size for width; default is 1
        s_h: stride size for heigth; default is 1
        
    Returns:
        out: output tensor of shape (batch_size, width_out, height_out, channel_out)
    """
    channel_in = x.get_shape()[-1]
    
    with tf.variable_scope(name):
        w = tf.get_variable('w', shape=[k_w, k_h, channel_in, channel_out], 
                            initializer=tf.truncated_normal_initializer(stddev=0.01))
        b = tf.get_variable('b', shape=[channel_out], initializer=tf.constant_initializer(0.0))
        
        out = tf.nn.conv2d(x, w, strides=[1, s_w, s_h, 1], padding='SAME') + b
        
        return out

In [3]:
def linear(x, dim_out, name=None):
    """Computes linear transform (fully-connected layer)
    
    Args:
        x: input tensor of shape (batch_size, dim_in)
        dim_out: dimension for output tensor
        
    Returns:
        out: output tensor of shape (batch_size, dim_out)
    """
    dim_in = x.get_shape()[-1]
    
    with tf.variable_scope(name):
        w = tf.get_variable('w', shape=[dim_in, dim_out], initializer=tf.truncated_normal_initializer(stddev=0.01))
        b = tf.get_variable('b', shape=[dim_out], initializer=tf.constant_initializer(0.0))
        
        out = tf.matmul(x, w) + b
        
        return out

In [4]:
def convolutional_network(x):
    # 1st convolution layer
    conv1 = conv2d(x, channel_out=8, name='conv1')
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 2nd convolution layer
    conv2 = conv2d(conv1, channel_out=16, name='conv2')
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # fully connected layer
    conv2 = tf.reshape(conv2, [-1, 7*7*16])
    h = linear(conv2, dim_out=64, name='hidden')
    h = tf.nn.relu(h)
    out = linear(h, dim_out=10, name='out')
    return out

## Construct Model

In [5]:
# construct model
x = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1])
y = tf.placeholder(dtype=tf.int64, shape=[None, 10])
out = convolutional_network(x)

## Loss, Optimizer and Summary

In [6]:
# loss 
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))

# accuracy
with tf.name_scope('accuracy'):
    pred = tf.argmax(out, 1)
    target = tf.argmax(y, 1)
    correct_pred = tf.equal(pred, target)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
# train op
with tf.name_scope('optimizer'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads_and_vars = list(zip(grads, tf.trainable_variables()))
    train_op = optimizer.apply_gradients(grads_and_vars=grads_and_vars)

# add summary op   
tf.scalar_summary('batch_loss', loss)
for var in tf.trainable_variables():
    tf.histogram_summary(var.op.name, var)
for grad, var in grads_and_vars:
    tf.histogram_summary(var.op.name+'/gradient', grad)

summary_op = tf.merge_all_summaries() 

## Train the Model

In [7]:
batch_size = 100
num_epoch = 5
num_iter_per_epoch = int(x_train.shape[0] / batch_size)
log_path = 'log/'

# launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize tensor variables
    tf.initialize_all_variables().run()
    summary_writer = tf.train.SummaryWriter(log_path, graph=tf.get_default_graph())
    # training phase
    for e in range(num_epoch):
        avg_loss = 0.
        # loop over all batches
        for i in range(num_iter_per_epoch):
            x_batch = x_train[i*batch_size:(i+1)*batch_size]
            y_batch = y_train[i*batch_size:(i+1)*batch_size]
            # run optimization op (backprop) and loss op (to get loss value)
            feed_dict={x: x_batch, y: y_batch}
            _, c = sess.run([train_op, loss], feed_dict=feed_dict)
            # compute average loss
            avg_loss += c / num_iter_per_epoch
            
            if i % 10 == 0:
                summary = sess.run(summary_op, feed_dict)
                summary_writer.add_summary(summary, e*num_iter_per_epoch + i)
        print ("Epoch %d, Loss: %.3f"% (e+1, avg_loss))
    print ("Finished training!\n")
    
    # test phase
    num_iter_per_epoch = int(x_test.shape[0] / batch_size)
    test_accuracy = 0.0
    for i in range(num_iter_per_epoch):
        x_batch = x_test[i*batch_size:(i+1)*batch_size]
        y_batch = y_test[i*batch_size:(i+1)*batch_size]
       
        acc = sess.run(accuracy, feed_dict={x: x_batch, y: y_batch})
        test_accuracy += acc

    print ("Test accuracy: %.3f" %(test_accuracy/num_iter_per_epoch))

Epoch 1, Loss: 1.128
Epoch 2, Loss: 0.148
Epoch 3, Loss: 0.087
Epoch 4, Loss: 0.066
Epoch 5, Loss: 0.054
Finished training!

Test accuracy: 0.972


<br>
## Execute TensorBoard
To execute the tensorboard, open the new terminal, run command below and open http://localhost:6005/ on your web browser.
```bash
$ tensorboard --logdir='./log' --port=6005 
```